# Import

In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.seasonal import STL

# Helper Functions

In [ ]:
def convert_date(date_str):
    try:
        date_obj = datetime.strptime(str(date_str), '%a %b %d %H:%M:%S %z %Y')
        return date_obj.strftime('%Y-%m-%d %H:%M:%S')
    except (ValueError, TypeError):
        return np.nan

def convert_date_df(df):
    df['date'] = df['created_at'].apply(convert_date)
    df = df.dropna(subset=['date'])
    df['date'] = pd.to_datetime(df['date'])

    # Create 'year' and 'month' columns
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    return df


def filter_emotion(df):
    df = df[['screen_name','joy_pys','sadness_pys','anger_pys','surprise_pys','disgust_pys','fear_pys','date']]
    df['date'] = pd.to_datetime(df['date'])
    return df

def filter_emotion_all(df):
    df = df[['screen_name','joy','sadness','anger','surprise','disgust','fear','date']]
    df['date'] = pd.to_datetime(df['date'])
    return df
def avg_emotion_per_user(df):
    df.loc[:, 'date'] = pd.to_datetime(df['date'])

    df.set_index('date', inplace=True)
    # Resample DataFrame to weekly frequency and calculate mean for each user
    weekly_avg_per_user = df.groupby('screen_name').resample('M').mean()#w
    # print(weekly_avg_per_user)

    # Calculate the overall weekly average by taking the mean of the weekly averages per user
    # overall_weekly_avg = weekly_avg_per_user.mean(level='date')
    overall_weekly_avg = weekly_avg_per_user.groupby(level='date').mean()

    return overall_weekly_avg


def replace_column_emotion(df):
    df.columns = df.columns.str.replace('_pys', '', regex=False)
    return df


# Load Data

In [ ]:
df = pd.read_csv('data_cross/owid-covid-data.csv')
usa = df[df['iso_code'] == "USA"]
usa_filtered = usa
#option to create the left graph
usa_filtered = usa[(usa['hosp_patients'] > 0) & (usa['new_cases'] > 0) & (usa['new_deaths'] > 0)]
usa_filtered = usa.dropna(subset=['hosp_patients', 'new_cases', 'new_deaths'])
usa_filtered = usa.dropna(subset=[ 'new_cases'])
start_date = '2020-01-26'
usa_filtered = usa_filtered[usa_filtered['date'] >= start_date]
usa_filtered


In [ ]:
max_date = '2022-05-15 00:00:00'

# Functions to create graphs

In [ ]:
import os


def pipline_analysis(final_h,final_df,usa,path, time_d,lags,divides_std,trend):
    if not os.path.exists(path):
        # If the directory does not exist, create it
        os.makedirs(path)
        print("Directory created:", path)


    hcp_avg_user = final_h.groupby('screen_name').resample(time_d).mean(numeric_only=True) # !!!!# Convert to comment
    general_avg_user = final_df.groupby('screen_name').resample(time_d).mean(numeric_only=True)# Convert to comment
    hcp_avg_user = hcp_avg_user.reset_index()# Convert to comment
    general_avg_user = general_avg_user.reset_index()# Convert to comment

    usa_copy = usa.copy()
    # usa_1 = usa_copy.dropna(subset=["hosp_patients"])
    usa_1 = usa_copy.dropna(subset=["new_cases"])

    usa_1['date'] = pd.to_datetime(usa_1['date'])
    min_date = usa_1['date'].min()

    overall_weekly_stats_h,max_date = calculate_df_per_tweet_limit(hcp_avg_user,min_date,time_d,divides_std,trend)# Convert to comment
    overall_weekly_stats_g,max_date = calculate_df_per_tweet_limit(general_avg_user,min_date, time_d,divides_std,trend)#0.30, 0.20 # Convert to comment
    # return overall_weekly_stats_h,max_date,overall_weekly_stats_g,max_date

    excel_corr_populations(usa, overall_weekly_stats_h, overall_weekly_stats_g, min_date, max_date,time_d,path,lags,divides_std,trend)
    excel_corr_data(usa,min_date, max_date,time_d,path,lags,divides_std,trend)

    path_img = path +"/correletio_between emotion_to_m"
    if not os.path.exists(path_img):
        # If the directory does not exist, create it
        os.makedirs(path_img)
    measures = ['new_cases', 'new_deaths','hosp_patients']#'hosp_patients_smoothed',
    for m in measures:
        analyze_correlations_between_em_me(usa, overall_weekly_stats_h, overall_weekly_stats_g, m, path_img,min_date,max_date,time_d,lags,divides_std,trend)

    emotions = ['joy', 'sadness', 'anger', 'surprise', 'disgust', 'fear']
    path_img = path +"/real_emotion_to_m"
    if not os.path.exists(path_img):
        # If the directory does not exist, create it
        os.makedirs(path_img)
    for m in measures:
        for emotion in emotions:

            graph_norm(overall_weekly_stats_h,overall_weekly_stats_g, m,emotion,time_d,path_img,min_date,max_date,divides_std,trend,usa)
            graph_norm_1(overall_weekly_stats_h,overall_weekly_stats_g, m,emotion,time_d,path_img,min_date,max_date,divides_std,trend,usa)




In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import t

def calculate_df_per_tweet_limit(df,min_date, grouped,divides_std,trend):
    # print(min_date)
    max_date = df['date'].max() - pd.Timedelta(days=21)
    df = df[(df['date'] >= min_date) & (df['date'] <= max_date)]

    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)

    # Select only numeric columns for quantile calculation
    numeric_df = df.select_dtypes(include=[np.number])

    overall_weekly_avg = numeric_df.resample(grouped).mean()
    overall_weekly_std = numeric_df.resample(grouped).std()
    overall_weekly_count = numeric_df.resample(grouped).count()

    # If divides_std is True, divide each column's mean by its std
    if divides_std:
        for col in numeric_df.columns:
            overall_weekly_avg[col] = overall_weekly_avg[col] / overall_weekly_std[col]

    # Rename the columns to reflect that these are averages possibly divided by std
    overall_weekly_avg.columns = ['avg_' + col for col in overall_weekly_avg.columns]


    # Concatenate the mean and confidence interval dataframes
    overall_weekly_stats = pd.concat([overall_weekly_avg], axis=1)

    return overall_weekly_stats,max_date


In [ ]:
def excel_corr_populations(usa, overall_weekly_stats_h, overall_weekly_stats_g, min_date, max_date,time_d,path,lags,divides_std,trend):
    measures = ['new_cases','hosp_patients','new_deaths']#'hosp_patients_smoothed',

    # Initialize list to store results
    table_data = []

    for measure in measures:

        # Analyze correlations
        result = analyze_correlations(usa, overall_weekly_stats_h, overall_weekly_stats_g, measure, min_date, max_date,time_d,lags,divides_std,trend)

        # Append results for healthcare professionals and non-healthcare professionals
        for idx, emotion in enumerate(result['hcp']['emotion']):
            table_data.append({
                'hcp/emotion': f'hcp-{emotion[4:]}',  # Strip 'avg_' from emotion name
                'emotion': emotion[4:],  # Add emotion name for sorting
                'measure': measure,
                'lag': result['hcp']['lag'][idx],
                'significance': result['hcp']['significance'][idx],
                'corr': result['hcp']['corr'][idx]
            })
            table_data.append({
                'hcp/emotion': f'non-hcp-{emotion[4:]}',  # Strip 'avg_' from emotion name
                'emotion': emotion[4:],  # Add emotion name for sorting
                'measure': measure,
                'lag': result['non_hcp']['lag'][idx],
                'significance': result['non_hcp']['significance'][idx],
                'corr': result['non_hcp']['corr'][idx]
            })

    # Convert results to DataFrame
    results_df = pd.DataFrame(table_data)
    results_df.to_csv(path+"/results_corr_finish.csv")

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import spearmanr

def analyze_correlations(usa, emotion_data_h, emotion_data_g, measure_column, min_date, max_date,group_d,lags,divied_s,trend):
    emotion_data_h.index = pd.to_datetime(emotion_data_h.index)
    emotion_data_g.index = pd.to_datetime(emotion_data_g.index)

    # Select the relevant columns (usa,measure_column, min_date, max_date,divied_s,trend,group_d)
    weekly_avg_daily_confirmed_cases = clean_usa(usa,measure_column, min_date, max_date,divied_s,trend,group_d)[measure_column]

    aggregate_emotions_h_df = emotion_data_h[['avg_joy', 'avg_sadness', 'avg_anger', 'avg_surprise', 'avg_disgust', 'avg_fear']]
    aggregate_emotions_g_df = emotion_data_g[['avg_joy', 'avg_sadness', 'avg_anger', 'avg_surprise', 'avg_disgust', 'avg_fear']]

    # Align the datasets to ensure they have the same length and the same dates
    aligned_data_h = pd.merge(weekly_avg_daily_confirmed_cases, aggregate_emotions_h_df, left_index=True, right_index=True, how='inner')
    aligned_data_g = pd.merge(weekly_avg_daily_confirmed_cases, aggregate_emotions_g_df, left_index=True, right_index=True, how='inner')

    # Define normalization function
    def normalize(x, range=(0, 1)):
        return (x - np.min(x)) / (np.max(x) - np.min(x)) * (range[1] - range[0]) + range[0]

    # Define function to calculate Spearman correlation with lag
    def spearmanr_with_lag(x, y, lag):
        if lag > 0:
            return spearmanr(x[:-lag], y[lag:])
        elif lag < 0:
            return spearmanr(x[-lag:], y[:lag])
        else:
            return spearmanr(x, y)

    # Calculate and plot correlations for both populations on the same graph
    emotion_list = ['avg_joy', 'avg_sadness', 'avg_anger', 'avg_surprise', 'avg_disgust', 'avg_fear']
    measure_name = f'weekly_avg_{measure_column}'

        # Initialize dictionaries to store results
    results_hcp = {'lag': [], 'significance': [], 'corr': [], 'emotion': []}
    results_non_hcp = {'lag': [], 'significance': [], 'corr': [], 'emotion': []}



    for emotion in emotion_list:
        # Normalize data
        # measure_data_h = normalize(aligned_data_h[measure_column])#######!!!!!!!!
        # emotion_data_normalized_h = normalize(aligned_data_h[emotion])#######!!!!!!!!
        # emotion_data_normalized_h = savgol_filter(aligned_data_h[emotion].tolist(), window_length=11, polyorder=2)
        measure_data_h = aligned_data_h[measure_column].tolist()
        emotion_data_normalized_h = aligned_data_h[emotion].tolist()


        # measure_data_g = normalize(aligned_data_g[measure_column])
        # emotion_data_normalized_g = normalize(aligned_data_g[emotion])

        measure_data_g = aligned_data_g[measure_column].tolist()
        emotion_data_normalized_g = aligned_data_g[emotion].tolist()
        # emotion_data_normalized_g = savgol_filter(aligned_data_g[emotion].tolist(), window_length=11, polyorder=2)

        if trend:

            stl = STL(emotion_data_normalized_h, seasonal=21).fit() #######################3new
            emotion_data_normalized_h = stl.trend

            stl = STL(emotion_data_normalized_g, seasonal=21).fit() #######################3new
            emotion_data_normalized_g = stl.trend

      # Initialize variables to store the best correlation results
        max_corr_h = 0  # For healthcare professionals
        best_pvalue_h = 0
        best_lag_h = 0

        max_corr_g = 0  # For non-healthcare professionals
        best_pvalue_g = 0
        best_lag_g = 0

        # Check correlations for different time displacements (lags)
        for displacement in lags:
            # Calculate Spearman correlation with lag for healthcare professionals
            corr_h = spearmanr_with_lag(measure_data_h, emotion_data_normalized_h, displacement)
            spearman_corr_h, p_value_h = corr_h[0], corr_h[1]

            # Update best correlation if current is greater
            if abs(max_corr_h)< abs(spearman_corr_h):
                max_corr_h = corr_h[0]
                best_pvalue_h = p_value_h
                best_lag_h = displacement

            # Calculate Spearman correlation with lag for non-healthcare professionals
            corr_g = spearmanr_with_lag(measure_data_g, emotion_data_normalized_g, displacement)
            spearman_corr_g, p_value_g = corr_g[0], corr_g[1]

            # Update best correlation if current is greater
            if abs(max_corr_g) < abs(spearman_corr_g):
                max_corr_g = corr_g[0]
                best_pvalue_g = p_value_g
                best_lag_g = displacement

        # Store the results for healthcare professionals
        results_hcp['lag'].append(best_lag_h)
        results_hcp['significance'].append(best_pvalue_h)
        results_hcp['corr'].append(max_corr_h)
        results_hcp['emotion'].append(emotion)

        # Store the results for non-healthcare professionals
        results_non_hcp['lag'].append(best_lag_g)
        results_non_hcp['significance'].append(best_pvalue_g)
        results_non_hcp['corr'].append(max_corr_g)
        results_non_hcp['emotion'].append(emotion)

    return {
        'hcp': results_hcp,
        'non_hcp': results_non_hcp
    }


In [ ]:
def clean_usa(usa,measure_column, min_date, max_date,divied_s,trend,group_d):
    usa_copy = usa.copy()
    usa = usa.dropna(subset=[measure_column])
    usa['date'] = pd.to_datetime(usa['date'])
    usa_filtered = usa[(usa['date'] >= min_date) & (usa['date'] <= max_date)]
    usa_filtered.set_index('date', inplace=True)
    numeric_columns = usa_filtered.select_dtypes(include=[np.number]).columns
    usa_weekly = usa_filtered[numeric_columns].resample(group_d).mean()
    usa_weekly_std = usa_filtered[numeric_columns].resample(group_d).std()


    return usa_weekly

In [ ]:
def excel_corr_data(usa,min_date, max_date,group_d,path,lags,divides_std,trend):
    path_img = path +"/imgs_corr_only_data_web"
    if not os.path.exists(path_img):
        # If the directory does not exist, create it
        os.makedirs(path_img)

    measures = [ 'new_cases','hosp_patients', 'new_deaths']#'hosp_patients_smoothed',

    # Initialize list to store results
    table_data = []

    for measure_1 in measures:
        for measure_2 in measures:
            if measure_2==measure_1:
                continue

        # Analyze correlations
            result = analyze_measure_correlations_data_from_web(usa, measure_1, measure_2, f't', min_date, max_date,group_d,lags,divides_std,trend)
            table_data.append([measure_2,measure_1,result[0],result[1],result[2]])
            analyze_correlations_data_web_im(usa,measure_1, measure_2 , path_img,min_date,max_date,lags,divides_std,group_d,trend)
        # Append results for healthcare professionals and non-healthcare professionals
    columns = ['measure_1','measure_2','corr','significance',"lag"]
    # Convert results to DataFrame
    results_df_new = pd.DataFrame(table_data,columns=columns)
    results_df_new.to_csv(path+"/results_corr_finish_data_usa.csv")

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import spearmanr

def analyze_correlations_between_em_me(usa, emotion_data_h, emotion_data_g, measure_column, output_path, min_date, max_date,time_d,lags,divides_std,trend):

    emotion_data_h.index = pd.to_datetime(emotion_data_h.index)
    emotion_data_g.index = pd.to_datetime(emotion_data_g.index)

    # if divides_std:
    #     usa_weekly = usa_weekly/usa_weekly_std

    # Select the relevant columns
    weekly_avg_daily_confirmed_cases = clean_usa(usa,measure_column, min_date, max_date,divides_std,trend,time_d)[measure_column]
    aggregate_emotions_h_df = emotion_data_h[['avg_joy', 'avg_sadness', 'avg_anger', 'avg_surprise', 'avg_disgust', 'avg_fear']]
    aggregate_emotions_g_df = emotion_data_g[['avg_joy', 'avg_sadness', 'avg_anger', 'avg_surprise', 'avg_disgust', 'avg_fear']]

    # Align the datasets to ensure they have the same length and the same dates
    aligned_data_h = pd.merge(weekly_avg_daily_confirmed_cases, aggregate_emotions_h_df, left_index=True, right_index=True, how='inner')
    aligned_data_g = pd.merge(weekly_avg_daily_confirmed_cases, aggregate_emotions_g_df, left_index=True, right_index=True, how='inner')

    # Define normalization function
    def normalize(x, range=(0, 1)):
        return (x - np.min(x)) / (np.max(x) - np.min(x)) * (range[1] - range[0]) + range[0]

    # Define function to calculate Spearman correlation with lag
    def spearmanr_with_lag(x, y, lag):
        if lag > 0:
            return spearmanr(x[:-lag], y[lag:])
        elif lag < 0:
            return spearmanr(x[-lag:], y[:lag])
        else:
            return spearmanr(x, y)

    # Calculate and plot correlations for both populations on the same graph
    emotion_list = ['avg_joy', 'avg_sadness', 'avg_anger', 'avg_surprise', 'avg_disgust', 'avg_fear']
    measure_name = f'weekly_avg_{measure_column}'
    results_data = []

    for emotion in emotion_list:
        # Normalize data



        measure_data_h = aligned_data_h[measure_column].tolist()
        emotion_data_normalized_h = aligned_data_h[emotion].tolist()
        measure_data_g = aligned_data_g[measure_column].tolist()
        emotion_data_normalized_g = aligned_data_g[emotion].tolist()
        if trend:
            stl = STL(emotion_data_normalized_h, seasonal=21).fit() #######################3new
            emotion_data_normalized_h = stl.trend

            stl = STL(emotion_data_normalized_g, seasonal=21).fit() #######################3new
            emotion_data_normalized_g = stl.trend

        corrs_h = []
        pvalues_h = []
        corrs_g = []
        pvalues_g = []

        for displacement in lags:
            corr_h = spearmanr_with_lag(measure_data_h, emotion_data_normalized_h, displacement)
            corrs_h.append(corr_h[0])
            pvalues_h.append(corr_h[1])
            corr_g = spearmanr_with_lag(measure_data_g, emotion_data_normalized_g, displacement)
            corrs_g.append(corr_g[0])
            pvalues_g.append(corr_g[1])
            results_data.append([emotion, displacement, corr_h[0], corr_h[1], corr_g[0], corr_g[1]])


        displacement = lags
        # Plot correlations on the same graph
        plt.figure(figsize=(30, 10))
        added_h_marker = False
        added_g_marker = False
        for i, pval in enumerate(pvalues_h):
            if pval < 0.05:
                plt.plot(displacement[i], corrs_h[i], '^', color='red', markersize=15, label='p < 0.05 HCPs' if not added_h_marker else "")
                added_h_marker = True

        for i, pval in enumerate(pvalues_g):
            if pval < 0.05:
                plt.plot(displacement[i], corrs_g[i], '^', color='blue', markersize=15, label='p < 0.05 Non-HCPs' if not added_g_marker else "")
                added_g_marker = True

        plt.plot(displacement, corrs_h, label='HCPs')
        plt.plot(displacement, corrs_g, label='Non-HCPs')

        # Add Spearman correlation values for corrs_h and corrs_g
        for i, (corr_h, corr_g) in enumerate(zip(corrs_h, corrs_g)):
            plt.text(displacement[i], corrs_h[i], f'{corr_h:.3f}', fontsize=26, ha='center', color='black', verticalalignment='bottom')
            plt.text(displacement[i], corrs_g[i], f'{corr_g:.3f}', fontsize=26, ha='center', color='black', verticalalignment='top')

        plt.title(f'Spearman correlation between {emotion.replace("avg_", "")} and {measure_name.replace("weekly_avg_", "").replace("_", " ")}', fontsize=35)
        plt.xlabel('Lags', fontsize=38)
        plt.ylabel('correlation coefficient', fontsize=38)
        # plt.legend(fontsize=16)  # Increase the font size of the legend

        # Increase the font size of the X and Y tick labels
        plt.xticks(fontsize=35)
        plt.yticks(fontsize=35)
        plt.legend(fontsize=22)
        plt.tight_layout()
        plt.savefig(output_path+ f'/correlation_between_{emotion}_and_{measure_name}.png')
    results_df = pd.DataFrame(results_data, columns=['Emotion', 'Lag', 'Correlation_HC', 'P-Value_HC', 'Correlation_NonHC', 'P-Value_NonHC'])
    results_df.to_excel(os.path.join(output_path, f'correlation{measure_column}_results.xlsx'), index=False)

        # plt.show()

# Example usage


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import spearmanr

def analyze_measure_correlations_data_from_web(usa, measure_column_1, measure_column_2, output_path, min_date, max_date,group_d,lags,divied_s,trend):


    usa_weekly_1 = clean_usa(usa,measure_column_1, min_date, max_date,divied_s,trend,group_d)[measure_column_1]
    usa_weekly_2 =  clean_usa(usa,measure_column_2, min_date, max_date,divied_s,trend,group_d)[measure_column_2]

    # Align the datasets
    aligned_data = pd.merge(usa_weekly_1, usa_weekly_2, left_index=True, right_index=True, how='inner')

    # Normalize the data
    def normalize(x):
        return (x - x.min()) / (x.max() - x.min()) if x.max() != x.min() else x


    # aligned_data[measure_column_1] = normalize(aligned_data[measure_column_1])
    # aligned_data[measure_column_2] = normalize(aligned_data[measure_column_2])  !!!!!!!!!!!!!!!!!!!!!!
    aligned_data[measure_column_1] = aligned_data[measure_column_1].tolist()
    aligned_data[measure_column_2] = aligned_data[measure_column_2].tolist()

    # Calculate Spearman correlation with lag
    max_corr_h = 0  # For healthcare professionals
    best_pvalue_h = 0
    best_lag_h = 0
    # lags = range(-8, 8)
    for lag in lags:
        if lag > 0:
            corr, pval = spearmanr(aligned_data[measure_column_1][:-lag], aligned_data[measure_column_2][lag:])
        elif lag < 0:
            corr, pval = spearmanr(aligned_data[measure_column_1][-lag:], aligned_data[measure_column_2][:lag])
        else:
            corr, pval = spearmanr(aligned_data[measure_column_1], aligned_data[measure_column_2])


        if abs(max_corr_h)< abs(corr):
                max_corr_h = corr
                best_pvalue_h = pval
                best_lag_h = lag



    return max_corr_h,best_pvalue_h,best_lag_h




In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
from scipy.signal import savgol_filter


def graph_norm(overall_weekly_stats_h,overall_weekly_stats_g, measure,emotion,group_d,path,min_date,max_date,divied_s,trend,usa):
    emotion_data_h = overall_weekly_stats_h

    # Reset index for emotion data
    emotion_data_h = emotion_data_h.reset_index()

    emotion_data_g = overall_weekly_stats_g.reset_index()


    usa_filtered = clean_usa(usa,measure, min_date, max_date,divied_s,trend,group_d)

    usa_filtered = usa_filtered.reset_index()
    # if measure == 'new_cases':
    #     print(usa_filtered[measure])
        # print(usa_weekly[measure])
    def normalize(x):
        return (x - x.min()) / (x.max() - x.min()) if x.max() != x.min() else x

    usa_filtered[f'{measure}_norm'] = normalize(usa_filtered[measure])
    emotion_data_h[f'avg_{emotion}_smooth'] = savgol_filter(emotion_data_h[f'avg_{emotion}'], window_length=11, polyorder=2)
    emotion_data_g[f'avg_{emotion}_smooth'] = savgol_filter(emotion_data_g[f'avg_{emotion}'], window_length=11, polyorder=2)

    emotion_data_h[f'avg_{emotion}_norm'] = normalize(emotion_data_h[f'avg_{emotion}_smooth']) #avg_fear !!!!!!!!!!!!!!!!!1
    emotion_data_g[f'avg_{emotion}_norm'] = normalize(emotion_data_g[f'avg_{emotion}_smooth'])



    if trend:
        stl = STL(emotion_data_h[f'avg_{emotion}_norm'], seasonal=21).fit() #######################3new
        emotion_data_h[f'avg_{emotion}_norm'] = stl.trend

        stl = STL(emotion_data_g[f'avg_{emotion}_norm'], seasonal=21).fit() #######################3new
        emotion_data_g[f'avg_{emotion}_norm'] = stl.trend
    measure_1 = measure
    plt.figure(figsize=(19, 6))
    if measure== "new_cases":
        measure_1 = "New Cases"

    # Plotting the measure data
    plt.plot(usa_filtered['date'], usa_filtered[f'{measure}_norm'], label=measure_1, color='red')
    plt.fill_between(usa_filtered['date'], usa_filtered[f'{measure}_norm'], color='red', alpha=0.3)


    # Plotting 'fear' data from emotion_data_h
    plt.plot(emotion_data_h['date'], emotion_data_h[f'avg_{emotion}_norm'], label=f'HCPs', color='blue')
    plt.plot(emotion_data_g['date'], emotion_data_g[f'avg_{emotion}_norm'], label=f'Non-HCPs', color='orange')
       # Get the minimum and maximum date from the datasets and set the x-axis limits
    min_date = min(emotion_data_h['date'].min(), emotion_data_g['date'].min())
    max_date = max(emotion_data_h['date'].max(), emotion_data_g['date'].max())
    plt.xlim(left=min_date, right=max_date)

    # Setting the labels and title
    plt.xlabel('Date', fontsize=22)
    plt.ylabel('Normalized Emotion Score', fontsize=22)
    if emotion== "joy":
        emotion = "Joy"
    if emotion == "fear":
        emotion= "Fear"
    if emotion == "sadness":
        emotion= "Sadness"


    plt.title(f'{measure_1.replace("_"," ")} VS. {emotion}', fontsize=20)

    # Adding grid and legend
    plt.grid(False)
    # plt.legend()

    # Set major ticks format and interval for the X-axis
    # plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=3))  # Display a date every 3 months
    # plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))  # Format dates as 'YYYY-MM'
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=1))  # Display a date every week
    # plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%y-%m'))
    plt.xticks(rotation=0,fontsize=14)

    plt.yticks(fontsize=17)
    plt.legend(fontsize=16,loc='best')
    plt.tight_layout()
    plt.savefig(path + f'/Comparison of COVID-19 {measure} and {emotion}.png')
    # Display the plot
    # plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import spearmanr

def analyze_correlations_data_web_im(usa, measure_column_1, measure_column_2, output_path, min_date, max_date,lags,divied_s,group_d,trend):
    usa_weekly_1 = clean_usa(usa,measure_column_1, min_date, max_date,divied_s,trend,group_d)[measure_column_1]
    usa_weekly_2 = clean_usa(usa,measure_column_2, min_date, max_date,divied_s,trend,group_d)[measure_column_2]

    # Align the datasets
    aligned_data = pd.merge(usa_weekly_1, usa_weekly_2, left_index=True, right_index=True, how='inner')

    # Normalize the data
    def normalize(x):
        return (x - x.min()) / (x.max() - x.min())

    # aligned_data[measure_column_1] = normalize(aligned_data[measure_column_1])   !!!!!!!!!!!!!!!!!!!!!!!!!!!!11
    # aligned_data[measure_column_2] = normalize(aligned_data[measure_column_2])

    aligned_data[measure_column_1] = aligned_data[measure_column_1].tolist()
    aligned_data[measure_column_2] = aligned_data[measure_column_2].tolist()

    # Calculate Spearman correlation with lag
    corrs = []
    pvalues = []
    # lags = range(-8, 9)
    for lag in lags:
        if lag > 0:
            corr, pval = spearmanr(aligned_data[measure_column_1][:-lag], aligned_data[measure_column_2][lag:])
        elif lag < 0:
            corr, pval = spearmanr(aligned_data[measure_column_1][-lag:], aligned_data[measure_column_2][:lag])
        else:
            corr, pval = spearmanr(aligned_data[measure_column_1], aligned_data[measure_column_2])
        corrs.append(corr)
        pvalues.append(pval)

    # Plot Spearman correlation results with conditional triangles
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(lags, corrs, label='Spearman Correlation')
    # Adding triangles for significant correlations
    for i, pval in enumerate(pvalues):
        if pval < 0.05:
            ax.plot(lags[i], corrs[i], '^', color='red', markersize=10, label='p < 0.05' if i == 0 else "")  # Add label only once

    ax.set_title(f'Spearman correlation between {measure_column_1} and {measure_column_2}')
    ax.set_xlabel('Lags',fontsize=17)
    ax.set_ylabel('correlation coefficient',fontsize=17)
    plt.yticks(fontsize=15)
    plt.xticks(fontsize=15)
    plt.legend()
    plt.tight_layout()
    plt.savefig(output_path+ f'/{measure_column_2}_{measure_column_1}.png')
    # plt.show()

In [ ]:

def graph_norm_1(overall_weekly_stats_h,overall_weekly_stats_g, measure,emotion,group_d,path,min_date,max_date,divied_s,trend,usa):
    emotion_data_h = overall_weekly_stats_h

    # Reset index for emotion data
    emotion_data_h = emotion_data_h.reset_index()

    emotion_data_g = overall_weekly_stats_g.reset_index()


    usa_filtered = clean_usa(usa,measure, min_date, max_date,divied_s,trend,group_d)

    usa_filtered = usa_filtered.reset_index()
    # if measure == 'new_cases':
    #     print(usa_filtered[measure])
        # print(usa_weekly[measure])
    def normalize(x):
        return (x - x.min()) / (x.max() - x.min()) if x.max() != x.min() else x

    usa_filtered[f'{measure}_norm'] = normalize(usa_filtered[measure])
    emotion_data_h[f'avg_{emotion}_smooth'] = savgol_filter(emotion_data_h[f'avg_{emotion}'], window_length=11, polyorder=2)
    emotion_data_g[f'avg_{emotion}_smooth'] = savgol_filter(emotion_data_g[f'avg_{emotion}'], window_length=11, polyorder=2)

    emotion_data_h[f'avg_{emotion}_norm'] = normalize(emotion_data_h[f'avg_{emotion}_smooth']) #avg_fear !!!!!!!!!!!!!!!!!1
    emotion_data_g[f'avg_{emotion}_norm'] = normalize(emotion_data_g[f'avg_{emotion}_smooth'])




    if trend:
        stl = STL(emotion_data_h[f'avg_{emotion}_norm'], seasonal=21).fit() #######################3new
        emotion_data_h[f'avg_{emotion}_norm'] = stl.trend

        stl = STL(emotion_data_g[f'avg_{emotion}_norm'], seasonal=21).fit() #######################3new
        emotion_data_g[f'avg_{emotion}_norm'] = stl.trend

    plt.figure(figsize=(18, 6))

    # Plotting the measure data
    plt.plot(usa_filtered['date'], usa_filtered[f'{measure}_norm'], label=measure, color='red')
    plt.fill_between(usa_filtered['date'], usa_filtered[f'{measure}_norm'], color='red', alpha=0.3)


    # Plotting 'fear' data from emotion_data_h
    plt.plot(emotion_data_h['date'], emotion_data_h[f'avg_{emotion}_norm'], label=f'HCPs', color='blue')
    # plt.plot(emotion_data_g['date'], emotion_data_g[f'avg_{emotion}_norm'], label=f'Non-HCPs', color='orange')

    # Setting the labels and title
    plt.xlabel('Date')
    plt.ylabel('Value')
    plt.title(f'Comparison of COVID-19 {measure} and {emotion}')

    # Adding grid and legend
    plt.grid(True)
    plt.legend()

    # Set major ticks format and interval for the X-axis
    # plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=3))  # Display a date every 3 months
    # plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))  # Format dates as 'YYYY-MM'
    plt.gca().xaxis.set_major_locator(mdates.WeekdayLocator(interval=1))  # Display a date every week
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.savefig(path + f'/Comparison of COVID-19 {measure} and {emotion}_only_hcp.png')
    # Display the plot
    # plt.show()


# Load Data

In [ ]:
# import pickle

# # Load HCP weekly stats
# with open("Fig3/weekly_stats_data_HCPs.pkl", "rb") as f:
#     overall_weekly_stats_h = pickle.load(f)

# # Load Non-HCP weekly stats
# with open("Fig3/weekly_stats_data_NonHCPs.pkl", "rb") as f:
#     overall_weekly_stats_g = pickle.load(f)


In [ ]:
overall_weekly_stats_h = pd.read_csv("weekly_stats_data_HCPs.csv", index_col="date", parse_dates=True)
overall_weekly_stats_g = pd.read_csv("weekly_stats_data_NonHCPs.csv", index_col="date", parse_dates=True)


In [ ]:
pipline_analysis(final_h,final_df,usa_filtered,"review_hcw_1", 'w',range(-8,9),False,False)#range(-56,57) (-8,9)


In [ ]:
cases = pd.read_csv('data_covid/new_cases.csv')
cases = cases[['date','United States']]
deaths = pd.read_csv('data_covid/new_deaths.csv')
deaths = deaths[['date','United States']]
hosp = pd.read_csv('data_covid/covid-hospitalizations.csv')
hosp_usa= hosp[hosp['iso_code'] == 'USA']
hosp_usa = hosp_usa[hosp_usa['indicator']=='Weekly new hospital admissions']

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

def normalize(x):
    """
    Normalize the input series by scaling between its min and max values.
    """
    return (x - x.min()) / (x.max() - x.min())

# Load data
cases = pd.read_csv('data_covid/new_cases.csv')
cases = cases[['date', 'United States']]
deaths = pd.read_csv('data_covid/new_deaths.csv')
deaths = deaths[['date', 'United States']]
hosp = pd.read_csv('data_covid/covid-hospitalizations.csv')

# Filter data for the USA
hosp_usa = hosp[(hosp['iso_code'] == 'USA') & (hosp['indicator'] == 'Weekly new hospital admissions')]

# Convert date column to datetime
cases['date'] = pd.to_datetime(cases['date'])
deaths['date'] = pd.to_datetime(deaths['date'])
hosp_usa['date'] = pd.to_datetime(hosp_usa['date'])

# Merge datasets on date
merged_df = pd.merge(cases, deaths, on='date', suffixes=('_cases', '_deaths'))
merged_df = pd.merge(merged_df, hosp_usa[['date', 'value']], on='date')

# Rename columns for easier reference
merged_df.rename(columns={'United States_cases': 'new_cases',
                          'United States_deaths': 'new_deaths',
                          'value': 'new_hospital_admissions'}, inplace=True)

# Normalize the data
merged_df['normalized_new_cases'] = normalize(merged_df['new_cases'])
merged_df['normalized_new_deaths'] = normalize(merged_df['new_deaths'])
merged_df['normalized_new_hospital_admissions'] = normalize(merged_df['new_hospital_admissions'])

# Plotting
plt.figure(figsize=(12, 6))
plt.plot(merged_df['date'], merged_df['normalized_new_cases'], label='New Cases', color='green')
# plt.plot(merged_df['date'], merged_df['normalized_new_deaths'], label='New Deaths', color='blue')
plt.plot(merged_df['date'], merged_df['normalized_new_hospital_admissions'], label='New Hospital Admissions', color='red')
plt.gca().xaxis.set_major_locator(mdates.WeekdayLocator(interval=4))  # Display a date every 4 weeks
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.xlabel('Date')
plt.ylabel('Normalized Data')
plt.title('COVID-19 New Cases, Deaths, and Hospital Admissions in the United States')
plt.grid(True)
plt.legend()

# Setting the date format on x-axis
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

def normalize(x):
    """
    Normalize the input series by scaling between its min and max values.
    """
    return (x - x.min()) / (x.max() - x.min())

# Load data
cases = pd.read_csv('data_covid/new_cases.csv')
cases = cases[['date', 'United States']]
deaths = pd.read_csv('data_covid/new_deaths.csv')
deaths = deaths[['date', 'United States']]
hosp = pd.read_csv('data_covid/covid-hospitalizations.csv')

# Filter data for the USA
hosp_usa = hosp[(hosp['iso_code'] == 'USA') & (hosp['indicator'] == 'Weekly new hospital admissions')]

# Convert date column to datetime
cases['date'] = pd.to_datetime(cases['date'])
deaths['date'] = pd.to_datetime(deaths['date'])
hosp_usa['date'] = pd.to_datetime(hosp_usa['date'])

# Merge datasets on date
merged_df = pd.merge(cases, deaths, on='date', suffixes=('_cases', '_deaths'))
merged_df = pd.merge(merged_df, hosp_usa[['date', 'value']], on='date', how='left')

# Rename columns for easier reference
merged_df.rename(columns={'United States_cases': 'new_cases',
                          'United States_deaths': 'new_deaths',
                          'value': 'new_hospital_admissions'}, inplace=True)

merged_df = merged_df[merged_df['date'] <= pd.to_datetime('2022-05-01')]

# Fill only the missing values in the middle with linear interpolation
if pd.isna(merged_df['new_hospital_admissions'].iloc[0]):
    merged_df['new_hospital_admissions'].iloc[0] = 0

# merged_df['new_hospital_admissions'] = merged_df['new_hospital_admissions'].interpolate(method='spline', order=3)

# Normalize the data
merged_df['normalized_new_cases'] = normalize(merged_df['new_cases'])
merged_df['normalized_new_deaths'] = normalize(merged_df['new_deaths'])
merged_df['normalized_new_hospital_admissions'] = normalize(merged_df['new_hospital_admissions'])

# Plotting
plt.figure(figsize=(12, 6))
plt.plot(merged_df['date'], merged_df['normalized_new_cases'], label='New Cases', color='green')
plt.plot(merged_df['date'], merged_df['normalized_new_deaths'], label='New Deaths', color='blue')
plt.plot(merged_df['date'], merged_df['normalized_new_hospital_admissions'], label='New Hospital Admissions', color='red')
plt.gca().xaxis.set_major_locator(mdates.WeekdayLocator(interval=4))  # Display a date every 4 weeks
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.xlabel('Date')
plt.ylabel('Normalized Data')
plt.title('COVID-19 New Cases, Deaths, and Hospital Admissions in the United States')
plt.grid(True)
plt.legend()

# Setting the date format on x-axis
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
